In [17]:
sc

In [4]:
%pylab inline

import pandas
import datetime as dt

from bigdl.nn.layer import *
from optparse import OptionParser
from bigdl.nn.criterion import *
from bigdl.optim.optimizer import *
from bigdl.util.common import *
from bigdl.dataset.transformer import *
from bigdl.nn.initialization_method import *
from transformer import *
from imagenet import *
#from bigdl.dataset import imagenet


Populating the interactive namespace from numpy and matplotlib


/Users/roshkhadka/anaconda/lib/python2.7/site-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['Normalize', 'random']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [5]:
# from bigdl.dataset import imagenet

In [6]:
def scala_T(input_T):
    if type(input_T) is list:
        # insert into index 0 spot, such that the real data starts from index 1
        temp = [0]
        temp.extend(input_T)
        return dict(enumerate(temp))
    # if dictionary, return it back
    return input_T

In [7]:
def Inception_Layer_v1(input_size, config, name_prefix=""):
    concat = Concat(2)
    conv1 = Sequential()
    conv1.add(
        SpatialConvolution(input_size,
                           config[1][1],
                           1, 1, 1, 1)
            .set_name(name_prefix + "1x1"))
    conv1.add(ReLU(True).set_name(name_prefix + "relu_1x1"))
    concat.add(conv1)
    conv3 = Sequential()
    conv3.add(SpatialConvolution(input_size, config[2][1], 1, 1, 1, 1)
              .set_name(name_prefix + "3x3_reduce"))
    conv3.add(ReLU(True).set_name(name_prefix + "relu_3x3_reduce"))
    conv3.add(SpatialConvolution(config[2][1], config[2][2],
                                 3, 3, 1, 1, 1, 1)
              .set_name(name_prefix + "3x3"))
    conv3.add(ReLU(True).set_name(name_prefix + "relu_3x3"))
    concat.add(conv3)
    conv5 = Sequential()
    conv5.add(SpatialConvolution(input_size,
                                 config[3][1], 1, 1, 1, 1)
              .set_name(name_prefix + "5x5_reduce"))
    conv5.add(ReLU(True).set_name(name_prefix + "relu_5x5_reduce"))
    conv5.add(SpatialConvolution(config[3][1],
                                 config[3][2], 5, 5, 1, 1, 2, 2)
              .set_name(name_prefix + "5x5"))
    conv5.add(ReLU(True).set_name(name_prefix + "relu_5x5"))
    concat.add(conv5)
    pool = Sequential()
    pool.add(SpatialMaxPooling(3, 3, 1, 1, 1, 1,
                               to_ceil=True).set_name(name_prefix + "pool"))
    pool.add(SpatialConvolution(input_size, config[4][1], 1, 1, 1, 1)
             .set_name(name_prefix + "pool_proj"))
    pool.add(ReLU(True).set_name(name_prefix + "relu_pool_proj"))
    concat.add(pool).set_name(name_prefix + "output")
    return concat

In [8]:
def Inception_v1_NoAuxClassifier(class_num):
    model = Sequential()
    model.add(SpatialConvolution(3, 64, 7, 7, 2, 2, 3, 3, 1,
                                 False).set_name("conv1/7x7_s2"))
    #---- correct API call---
    #model.add(SpatialConvolution(3, 64, 7, 7, 2, 2, 3, 3, 1,
    #                             False).set_init_method(weight_init_method = Xavier()).set_name("conv1/7x7_s2"))
    model.add(ReLU(True).set_name("conv1/relu_7x7"))
    model.add(SpatialMaxPooling(3, 3, 2, 2, to_ceil=True).set_name("pool1/3x3_s2"))
    model.add(SpatialCrossMapLRN(5, 0.0001, 0.75).set_name("pool1/norm1"))
    model.add(SpatialConvolution(64, 64, 1, 1, 1, 1)
              .set_name("conv2/3x3_reduce"))
    model.add(ReLU(True).set_name("conv2/relu_3x3_reduce"))
    model.add(SpatialConvolution(64, 192, 3, 3, 1, 1, 1, 1)
              .set_name("conv2/3x3"))
    model.add(ReLU(True).set_name("conv2/relu_3x3"))
    model.add(SpatialCrossMapLRN(5, 0.0001, 0.75).set_name("conv2/norm2"))
    model.add(SpatialMaxPooling(3, 3, 2, 2, to_ceil=True).set_name("pool2/3x3_s2"))
    model.add(Inception_Layer_v1(192, scala_T([scala_T([64]), scala_T(
        [96, 128]), scala_T([16, 32]), scala_T([32])]), "inception_3a/"))
    model.add(Inception_Layer_v1(256, scala_T([scala_T([128]), scala_T(
        [128, 192]), scala_T([32, 96]), scala_T([64])]), "inception_3b/"))
    model.add(SpatialMaxPooling(3, 3, 2, 2, to_ceil=True))
    model.add(Inception_Layer_v1(480, scala_T([scala_T([192]), scala_T(
        [96, 208]), scala_T([16, 48]), scala_T([64])]), "inception_4a/"))
    model.add(Inception_Layer_v1(512, scala_T([scala_T([160]), scala_T(
        [112, 224]), scala_T([24, 64]), scala_T([64])]), "inception_4b/"))
    model.add(Inception_Layer_v1(512, scala_T([scala_T([128]), scala_T(
        [128, 256]), scala_T([24, 64]), scala_T([64])]), "inception_4c/"))
    model.add(Inception_Layer_v1(512, scala_T([scala_T([112]), scala_T(
        [144, 288]), scala_T([32, 64]), scala_T([64])]), "inception_4d/"))
    model.add(Inception_Layer_v1(528, scala_T([scala_T([256]), scala_T(
        [160, 320]), scala_T([32, 128]), scala_T([128])]), "inception_4e/"))
    model.add(SpatialMaxPooling(3, 3, 2, 2, to_ceil=True))
    model.add(Inception_Layer_v1(832, scala_T([scala_T([256]), scala_T(
        [160, 320]), scala_T([32, 128]), scala_T([128])]), "inception_5a/"))
    model.add(Inception_Layer_v1(832, scala_T([scala_T([384]), scala_T(
        [192, 384]), scala_T([48, 128]), scala_T([128])]), "inception_5b/"))
    model.add(SpatialAveragePooling(7, 7, 1, 1).set_name("pool5/7x7_s1"))
    model.add(Dropout(0.4).set_name("pool5/drop_7x7_s1"))
    model.add(View([1024], num_input_dims=3))
    model.add(Linear(1024, class_num).set_name("loss3/classifier"))
    model.add(LogSoftMax().set_name("loss3/loss3"))
    model.reset()
    return model


In [18]:
def Inception_v1(class_num):
    feature1 = Sequential()
    feature1.add(SpatialConvolution(3, 64, 7, 7, 2, 2, 3, 3, 1, False)
                 .set_name("conv1/7x7_s2"))
    feature1.add(ReLU(True).set_name("conv1/relu_7x7"))
    feature1.add(
        SpatialMaxPooling(3, 3, 2, 2, to_ceil=True)
            .set_name("pool1/3x3_s2"))
    feature1.add(SpatialCrossMapLRN(5, 0.0001, 0.75)
                 .set_name("pool1/norm1"))
    feature1.add(SpatialConvolution(64, 64, 1, 1, 1, 1)
                 .set_name("conv2/3x3_reduce"))
    feature1.add(ReLU(True).set_name("conv2/relu_3x3_reduce"))
    feature1.add(SpatialConvolution(64, 192, 3, 3, 1, 1, 1, 1).set_name("conv2/3x3"))
    
    feature1.add(ReLU(True).set_name("conv2/relu_3x3"))
    feature1.add(SpatialCrossMapLRN(5, 0.0001, 0.75).set_name("conv2/norm2"))
    feature1.add(
        SpatialMaxPooling(3, 3, 2, 2, to_ceil=True).set_name("pool2/3x3_s2"))
    feature1.add(Inception_Layer_v1(192,
                                    scala_T([scala_T([64]), scala_T([96, 128]),
                                             scala_T([16, 32]), scala_T([32])]),
                                    "inception_3a/"))
    feature1.add(Inception_Layer_v1(256, scala_T([
        scala_T([128]), scala_T([128, 192]), scala_T([32, 96]), scala_T([64])]),
                                    "inception_3b/"))
    feature1.add(
        SpatialMaxPooling(3, 3, 2, 2, to_ceil=True).set_name("pool3/3x3_s2"))
    feature1.add(Inception_Layer_v1(480, scala_T([
        scala_T([192]), scala_T([96, 208]), scala_T([16, 48]), scala_T([64])]),
                                    "inception_4a/"))

    output1 = Sequential()
    output1.add(SpatialAveragePooling(
        5, 5, 3, 3, ceil_mode=True).set_name("loss1/ave_pool"))
    output1.add(
        SpatialConvolution(512, 128, 1, 1, 1, 1).set_name("loss1/conv"))
    output1.add(ReLU(True).set_name("loss1/relu_conv"))
    output1.add(View([128 * 4 * 4, 3]))
    output1.add(Linear(128 * 4 * 4, 1024).set_name("loss1/fc"))
    output1.add(ReLU(True).set_name("loss1/relu_fc"))
    output1.add(Dropout(0.7).set_name("loss1/drop_fc"))
    output1.add(Linear(1024, class_num).set_name("loss1/classifier"))
    output1.add(LogSoftMax().set_name("loss1/loss"))

    feature2 = Sequential()
    feature2.add(Inception_Layer_v1(512,
                                    scala_T([scala_T([160]), scala_T([112, 224]),
                                             scala_T([24, 64]), scala_T([64])]),
                                    "inception_4b/"))
    feature2.add(Inception_Layer_v1(512,
                                    scala_T([scala_T([128]), scala_T([128, 256]),
                                             scala_T([24, 64]), scala_T([64])]),
                                    "inception_4c/"))
    feature2.add(Inception_Layer_v1(512,
                                    scala_T([scala_T([112]), scala_T([144, 288]),
                                             scala_T([32, 64]), scala_T([64])]),
                                    "inception_4d/"))

    output2 = Sequential()
    output2.add(SpatialAveragePooling(5, 5, 3, 3).set_name("loss2/ave_pool"))
    output2.add(
        SpatialConvolution(528, 128, 1, 1, 1, 1).set_name("loss2/conv"))
    output2.add(ReLU(True).set_name("loss2/relu_conv"))
    output2.add(View([128 * 4 * 4, 3]))
    output2.add(Linear(128 * 4 * 4, 1024).set_name("loss2/fc"))
    output2.add(ReLU(True).set_name("loss2/relu_fc"))
    output2.add(Dropout(0.7).set_name("loss2/drop_fc"))
    output2.add(Linear(1024, class_num).set_name("loss2/classifier"))
    output2.add(LogSoftMax().set_name("loss2/loss"))

    output3 = Sequential()
    output3.add(Inception_Layer_v1(528,
                                   scala_T([scala_T([256]), scala_T([160, 320]),
                                            scala_T([32, 128]), scala_T([128])]),
                                   "inception_4e/"))
    output3.add(SpatialMaxPooling(3, 3, 2, 2, to_ceil=True).set_name("pool4/3x3_s2"))
    output3.add(Inception_Layer_v1(832,
                                   scala_T([scala_T([256]), scala_T([160, 320]),
                                            scala_T([32, 128]), scala_T([128])]),
                                   "inception_5a/"))
    output3.add(Inception_Layer_v1(832,
                                   scala_T([scala_T([384]), scala_T([192, 384]),
                                            scala_T([48, 128]), scala_T([128])]),
                                   "inception_5b/"))
    output3.add(SpatialAveragePooling(7, 7, 1, 1).set_name("pool5/7x7_s1"))
    output3.add(Dropout(0.4).set_name("pool5/drop_7x7_s1"))
    output3.add(View([1024, 3]))
    output3.add(Linear(1024, class_num)
                .set_name("loss3/classifier"))
    output3.add(LogSoftMax().set_name("loss3/loss3"))

    split2 = Concat(2).set_name("split2")
    split2.add(output3)
    split2.add(output2)

    mainBranch = Sequential()
    mainBranch.add(feature2)
    mainBranch.add(split2)

    split1 = Concat(2).set_name("split1")
    split1.add(mainBranch)
    split1.add(output1)

    model = Sequential()

    model.add(feature1)
    model.add(split1)

    model.reset()
    return model


In [19]:
def get_inception_data(folder, file_type="image", data_type="train", normalize=255.0):
    path = os.path.join(folder, data_type)
    if "seq" == file_type:
        # return imagenet.read_seq_file(sc, path, normalize)
        return read_seq_file(sc, path, normalize)
    elif "image" == file_type:
        #return imagenet.read_local(sc, path, normalize)
        return read_local(sc, path, normalize)

In [20]:
def config_option_parser():
    parser = OptionParser()
    
    parser.add_option("-a", "--action", dest="action", default="train")
    #parser.add_option("-f", "--folder", type=str, dest="folder", default="",
    #                  help="url of hdfs folder store the hadoop sequence files")
    
    parser.add_option("-f", "--folder", type=str, dest="folder", default="/home/rocky/git/datasets/notfruit/fruit_dataset/sample",
                      help="url of hdfs folder store the hadoop sequence files")
                        
    parser.add_option("--model", type=str, dest="model", default="", help="model snapshot location")
    parser.add_option("--state", type=str, dest="state", default="", help="state snapshot location")
    parser.add_option("--checkpoint", type=str, dest="checkpoint", default="", help="where to cache the model")
    parser.add_option("-o", "--overwrite", action="store_true", dest="overwrite", default=False,
                      help="overwrite checkpoint files")
    parser.add_option("-e", "--maxEpoch", type=int, dest="maxEpoch", default=0, help="epoch numbers")
    parser.add_option("-i", "--maxIteration", type=int, dest="maxIteration", default=62000, help="iteration numbers")
    parser.add_option("-l", "--learningRate", type=float, dest="learningRate", default=0.01, help="iteration numbers")
    parser.add_option("-b", "--batchSize", type=int, dest="batchSize",default = 64, help="batch size")
    parser.add_option("--classNum", type=int, dest="classNum", default=1000, help="class number")
    parser.add_option("--weightDecay", type=float, dest="weightDecay", default=0.0001, help="weight decay")
    parser.add_option("--checkpointIteration", type=int, dest="checkpointIteration", default=620,
                      help="checkpoint interval of iterations")

    return parser

In [21]:
if __name__ == "__main__":     
    # parse options

    parser = config_option_parser()
    (options, args) = parser.parse_args(sys.argv)

    if not options.learningRate:
        parser.error("-l --learningRate is a mandatory opt")
    if not options.batchSize:
        parser.error("-b --batchSize is a mandatory opt")
        
    #learningRate = 0.001    imagenet

    # init
    #sc = SparkContext(appName="inception v1", conf=create_spark_conf())
    init_engine()
    # build model
#     inception_model = Inception_v1_NoAuxClassifier(options.classNum)
    inception_model = Inception_v1(options.classNum)

    image_size = 224
    options.action = "train"
    if options.action == "train":
        # create dataset
        train_transformer = Transformer([Crop(image_size, image_size),
                                         Flip(0.5),
                                         ChannelNormalizer(0.485, 0.456, 0.406, 0.229, 0.224, 0.225),
                                         TransposeToTensor(False)
                                         ])
        #train_data = get_inception_data(options.folder, "seq", "train").map(
        #options.folder = "/home/rocky/git/datasets/notfruit/fruit_dataset/sample/"
        options.folder = "./sample"

        train_data = get_inception_data(options.folder, "image", "train").map(
            lambda features_label: (train_transformer(features_label[0]), features_label[1])).map(
            lambda features_label: Sample.from_ndarray(features_label[0], features_label[1] + 1))

        val_transformer = Transformer([Crop(image_size, image_size, "center"),
                                       Flip(0.5),
                                       ChannelNormalizer(0.485, 0.456, 0.406, 0.229, 0.224, 0.225),
                                       TransposeToTensor(False)
                                       ])
        #val_data = get_inception_data(options.folder, "seq", "val").map(
        val_data = get_inception_data(options.folder, "image", "val").map(
            lambda features_label: (val_transformer(features_label[0]), features_label[1])).map(
            lambda features_label: Sample.from_ndarray(features_label[0], features_label[1] + 1))

        # TODO: Check stateSnapshot opt

        if options.maxEpoch:
                {"learningRate": options.learningRate,
                 "weightDecay": options.weightDecay,
                 "momentum": 0.9,
                 "dampening": 0.0}
                checkpoint_trigger = EveryEpoch()
                test_trigger = EveryEpoch()
                end_trigger = MaxEpoch(options.maxEpoch)
        else:
            state = scala_T(
                {"learningRate": options.learningRate,
                 "weightDecay": options.weightDecay,
                 "momentum": 0.9,
                 "dampening": 0.0})
            checkpoint_trigger = SeveralIteration(options.checkpointIteration)
            test_trigger = SeveralIteration(options.checkpointIteration)
            end_trigger = MaxIteration(options.maxIteration)

        # Optimizer
        optimizer = Optimizer(
            model=inception_model,
            training_rdd=train_data,
            #optim_method="SGD",
            #optim_method=SGD(learningrate=0.01, learningrate_decay=0.0002)
            optim_method=Adam(learningrate=0.002),
            criterion=ClassNLLCriterion(),
            end_trigger=end_trigger,
            batch_size=options.batchSize,
            #state=state
        )
        options.batchSize = 64 
        if options.checkpoint:
            optimizer.set_checkpoint(checkpoint_trigger, options.checkpoint, options.overwrite)

        """
        optimizer.set_validation(trigger=test_trigger,
                                 val_rdd=val_data,
                                 batch_size=options.batchSize,
                                 #val_method=["Top1Accuracy", "Top5Accuracy"])
                                 val_method=["Loss"]
                                 val_method=[Top1Accuracy()])
        """
        optimizer.set_validation(batch_size=2048,
                                val_rdd=val_data,
                                trigger=EveryEpoch(),
                                val_method=[Top1Accuracy()])

        trained_model = optimizer.optimize()


    options.action = "test"
    if options.action == "test":
        # Load a pre-trained model and then validate it through top1 accuracy.
        test_transformer = Transformer([Crop(image_size, image_size, "center"),
                                        Flip(0.5),
                                        ChannelNormalizer(0.485, 0.456, 0.406, 0.229, 0.224, 0.225),
                                        TransposeToTensor(False)
                                        ])
        #test_data = get_inception_data(options.folder, "", "val").map(
        test_data = get_inception_data(options.folder, "image", "val").map(
            lambda features_label: (test_transformer(features_label[0]), features_label[1])).map(
            lambda features_label: Sample.from_ndarray(features_label[0], features_label[1] + 1))
        model = Model.load(options.model)
        results = model.test(test_data, options.batchSize, ["Top1Accuracy", "Top5Accuracy"])
        for result in results:
            print result

    print ("---- the end -----")

creating: createSequential
creating: createSpatialConvolution
creating: createReLU
creating: createSpatialMaxPooling
creating: createSpatialCrossMapLRN
creating: createSpatialConvolution
creating: createReLU
creating: createSpatialConvolution
creating: createReLU
creating: createSpatialCrossMapLRN
creating: createSpatialMaxPooling
creating: createConcat
creating: createSequential
creating: createSpatialConvolution
creating: createReLU
creating: createSequential
creating: createSpatialConvolution
creating: createReLU
creating: createSpatialConvolution
creating: createReLU
creating: createSequential
creating: createSpatialConvolution
creating: createReLU
creating: createSpatialConvolution
creating: createReLU
creating: createSequential
creating: createSpatialMaxPooling
creating: createSpatialConvolution
creating: createReLU
creating: createConcat
creating: createSequential
creating: createSpatialConvolution
creating: createReLU
creating: createSequential
creating: createSpatialConvolutio

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/Users/roshkhadka/Desktop/BigdL_0.2.0/setup/spark-2.1.0-bin-hadoop2.7.3/python/lib/py4j-0.10.4-src.zip/py4j/java_gateway.py", line 883, in send_command
    response = connection.send_command(command)
  File "/Users/roshkhadka/Desktop/BigdL_0.2.0/setup/spark-2.1.0-bin-hadoop2.7.3/python/lib/py4j-0.10.4-src.zip/py4j/java_gateway.py", line 1040, in send_command
    "Error while receiving", e, proto.ERROR_ON_RECEIVE)
Py4JNetworkError: Error while receiving
Traceback (most recent call last):
  File "/Users/roshkhadka/anaconda/lib/python2.7/SocketServer.py", line 290, in _handle_request_noblock


----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 60227)


    self.process_request(request, client_address)
  File "/Users/roshkhadka/anaconda/lib/python2.7/SocketServer.py", line 318, in process_request
    self.finish_request(request, client_address)
  File "/Users/roshkhadka/anaconda/lib/python2.7/SocketServer.py", line 331, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/Users/roshkhadka/anaconda/lib/python2.7/SocketServer.py", line 652, in __init__
    self.handle()
  File "/Users/roshkhadka/Desktop/BigdL_0.2.0/setup/spark-2.1.0-bin-hadoop2.7.3/python/pyspark/accumulators.py", line 235, in handle
ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/Users/roshkhadka/Desktop/BigdL_0.2.0/setup/spark-2.1.0-bin-hadoop2.7.3/python/lib/py4j-0.10.4-src.zip/py4j/java_gateway.py", line 883, in send_command
    response = connection.send_command(command)
  File "/Users/roshkhadka/Desktop/BigdL_0.2.0/setup/spark-2.1.0-bin-hadoop2.7.3/python/lib/py4j-0.10.4-src.zip/py4j/java_ga

----------------------------------------


    num_updates = read_int(self.rfile)
  File "/Users/roshkhadka/Desktop/BigdL_0.2.0/setup/spark-2.1.0-bin-hadoop2.7.3/python/pyspark/serializers.py", line 557, in read_int
    raise EOFError
EOFError


Py4JError: An error occurred while calling None.None

In [ ]:
print 'hello'